In [25]:
# Init
# choropleth
from __future__ import division
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import matplotlib.pyplot as plt
import seaborn as sns

# County level data retrieval

In [26]:
df = pd.read_csv('../data/2019_data/prepped_data_2019.csv', na_values=".")
print(df.columns.tolist())

['Unnamed: 0', 'Unnamed: 0_x', 'fips', 'Geo_NAME', 'Geo_QName', 'Geo_STUSAB', 'Geo_SUMLEV', 'Geo_GEOCOMP', 'Geo_FILEID', 'Geo_LOGRECNO', 'Geo_US', 'Geo_REGION', 'Geo_DIVISION', 'Geo_STATECE', 'Geo_STATE', 'Geo_COUNTY', 'Geo_COUSUB', 'Geo_PLACE', 'Geo_PLACESE', 'Geo_TRACT', 'Geo_BLKGRP', 'Geo_CONCIT', 'Geo_AIANHH', 'Geo_AIANHHFP', 'Geo_AIHHTLI', 'Geo_AITSCE', 'Geo_AITS', 'Geo_ANRC', 'Geo_CBSA', 'Geo_CSA', 'Geo_METDIV', 'Geo_MACC', 'Geo_MEMI', 'Geo_NECTA', 'Geo_CNECTA', 'Geo_NECTADIV', 'Geo_UA', 'Geo_UACP', 'Geo_CDCURR', 'Geo_SLDU', 'Geo_SLDL', 'Geo_VTD', 'Geo_ZCTA3', 'Geo_ZCTA5', 'Geo_SUBMCD', 'Geo_SDELM', 'Geo_SDSEC', 'Geo_SDUNI', 'Geo_UR', 'Geo_PCI', 'Geo_TAZ', 'Geo_UGA', 'Geo_PUMA5', 'Geo_PUMA1', 'Geo_GEOID', 'Geo_BTTR', 'Geo_BTBG', 'Geo_AREALAND', 'Geo_AREAWATR', 'ACS19_5yr_B01001001', 'ACS19_5yr_B01001002', 'ACS19_5yr_B01001003', 'ACS19_5yr_B01001004', 'ACS19_5yr_B01001005', 'ACS19_5yr_B01001006', 'ACS19_5yr_B01001007', 'ACS19_5yr_B01001008', 'ACS19_5yr_B01001009', 'ACS19_5yr_B0100

# Data Cleanup involves combining data from two different years

In [27]:
# ##### Data Cleanup

# df = df.dropna(subset=['vehicles_per_worker',
#                                  'total_workers',
#                                  'land_area',
#                                  'pct_unemployed',
#                                  'average_commute_time_per_worker_drive_alone',
#                                  'pct_car_wrkr_cmute_01_to_14_min',
#                                  'pct_car_wrkr_cmute_15_to_29_min',
#                                  'pct_car_wrkr_cmute_30_to_44_min',
#                                  'pct_car_wrkr_cmute_44_to_59_min',
#                                  'pct_car_wrkr_cmute_over_60_min',
#                                  'total_population'])

# df['total_workers'] = df['total_workers']
# df['land_area'] = df['land_area']
# df['vehicles_per_worker'] = df['vehicles_per_worker']
# df['average_commute_time_per_worker_drive_alone'] = df['average_commute_time_per_worker_drive_alone']
# df['pct_car_wrkr_cmute_01_to_14_min'] = df['pct_car_wrkr_cmute_01_to_14_min']
# df['pct_car_wrkr_cmute_15_to_29_min'] = df['pct_car_wrkr_cmute_15_to_29_min']
# df['pct_car_wrkr_cmute_44_to_59_min'] = df['pct_car_wrkr_cmute_44_to_59_min']
# df['pct_car_wrkr_cmute_30_to_44_min'] = df['pct_car_wrkr_cmute_30_to_44_min']
# df['pct_car_wrkr_cmute_over_60_min'] = df['pct_car_wrkr_cmute_over_60_min']
# df['pct_unemployed'] = df['pct_unemployed']
# df['total_population'] = df['total_population']
# df['year'] = 2019

# OLS Regression - Multivariate

In [28]:
df = df
df.pct_walking_and_public_transit = df.pct_all_public_tspt_wrkr_cmute + df.pct_walked_wrkr_cmute
df.pct_walking_and_public_transit


C:\Users\cjong\AppData\Local\Temp\ipykernel_2340\3390946073.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.pct_walking_and_public_transit = df.pct_all_public_tspt_wrkr_cmute + df.pct_walked_wrkr_cmute


0       0.015675
1       0.006936
2       0.030587
3       0.001391
4       0.005852
          ...   
3215    0.019362
3216    0.044232
3217    0.025020
3218    0.020998
3219    0.038529
Length: 3220, dtype: float64

In [29]:
df = df
df['pct_walking_and_public_transit'] = df['pct_all_public_tspt_wrkr_cmute'] + df['pct_walked_wrkr_cmute']
df['pct_walking_and_public_transit']


0       0.015675
1       0.006936
2       0.030587
3       0.001391
4       0.005852
          ...   
3215    0.019362
3216    0.044232
3217    0.025020
3218    0.020998
3219    0.038529
Name: pct_walking_and_public_transit, Length: 3220, dtype: float64

In [33]:
lm_ols = smf.ols(formula = 'pct_walking_and_public_transit ~ \
                                vehicles_per_worker + \
                                roads_length + \
                                intersections + \
                                total_population + \
                                average_commute_time_per_worker_drive_alone + \
                                pct_unemployed + \
                                land_area + \
                                cbp_total_employees + \
                                cbp_total_establishments',
data = df).fit()
print (lm_ols.summary())

                                  OLS Regression Results                                  
Dep. Variable:     pct_walking_and_public_transit   R-squared:                       0.770
Model:                                        OLS   Adj. R-squared:                  0.769
Method:                             Least Squares   F-statistic:                     513.2
Date:                            Fri, 16 Dec 2022   Prob (F-statistic):               0.00
Time:                                    15:06:13   Log-Likelihood:                 2886.2
No. Observations:                            1389   AIC:                            -5752.
Df Residuals:                                1379   BIC:                            -5700.
Df Model:                                       9                                         
Covariance Type:                        nonrobust                                         
                                                  coef    std err          t      P>|t|   

In [37]:
lm_ols = smf.ols(formula = 'pct_walking_and_public_transit ~ \
                                vehicles_per_worker + \
                                roads_length + \
                                intersections + \
                                total_population + \
                                pct_unemployed + \
                                land_area + \
                                cbp_total_employees + \
                                cbp_total_establishments \
                                ',
                                # average_commute_time_per_worker_drive_alone + \
data = df).fit()
print (lm_ols.summary())

                                  OLS Regression Results                                  
Dep. Variable:     pct_walking_and_public_transit   R-squared:                       0.698
Model:                                        OLS   Adj. R-squared:                  0.697
Method:                             Least Squares   F-statistic:                     904.9
Date:                            Fri, 16 Dec 2022   Prob (F-statistic):               0.00
Time:                                    17:09:36   Log-Likelihood:                 6493.6
No. Observations:                            3139   AIC:                        -1.297e+04
Df Residuals:                                3130   BIC:                        -1.291e+04
Df Model:                                       8                                         
Covariance Type:                        nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.97

In [38]:
lm_ols = smf.ols(formula = 'pct_walking_and_public_transit ~ \
                                vehicles_per_worker + \
                                roads_length + \
                                intersections + \
                                total_population + \
                                pct_unemployed + \
                                land_area ',
                                # average_commute_time_per_worker_drive_alone + \
data = df).fit()
print (lm_ols.summary())

                                  OLS Regression Results                                  
Dep. Variable:     pct_walking_and_public_transit   R-squared:                       0.696
Model:                                        OLS   Adj. R-squared:                  0.696
Method:                             Least Squares   F-statistic:                     1198.
Date:                            Fri, 16 Dec 2022   Prob (F-statistic):               0.00
Time:                                    17:10:10   Log-Likelihood:                 6463.1
No. Observations:                            3141   AIC:                        -1.291e+04
Df Residuals:                                3134   BIC:                        -1.287e+04
Df Model:                                       6                                         
Covariance Type:                        nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
--

# LASSO REGRESSION

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
# from sklearn.linear_model import ElasticNet
# model = ElasticNet(alpha=1.0, l1_ratio=0.5)
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()

# X = df['pct_walking_and_public_transit']
# y = df[['vehicles_per_worker','roads_length','intersections','total_population','average_commute_time_per_worker_drive_alone','pct_unemployed','land_area']]

df = df.dropna(subset=['pct_walking_and_public_transit','vehicles_per_worker','roads_length','intersections','total_population','average_commute_time_per_worker_drive_alone','pct_unemployed','land_area','cbp_total_employees', 'cbp_total_establishments'])

X = df[['vehicles_per_worker','roads_length','intersections','total_population','average_commute_time_per_worker_drive_alone','pct_unemployed','land_area','cbp_total_employees', 'cbp_total_establishments']]
y = df['pct_walking_and_public_transit']

In [41]:

# Here, I standardize by X data using StandardScalar
X_train, X_test, y_train, y_test = train_test_split(X, y)
# scaler.fit(X_test)
# X_test_scaled = scaler.transform(X_test)
# scaler.fit(X_train)
# X_train_scaled = scaler.transform(X_train)

scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [42]:
lasso = Lasso().fit(X_train, y_train) # Fit the training data to a lasso regression line with alpha = 0.01 and 100,000 iterations

print("LASSO REGRESSION (UNSCALED DATA)") # LASSO is L1 penalty, which adds “absolute value of magnitude” of coefficient as penalty term to the loss function. Good for feature selection when you have a lot of features.
print("Training set score: {:.3f}".format(lasso.score(X_train, y_train)))
print("Test set score: {:.3f}".format(lasso.score(X_test, y_test)))

# Kfold cross validation
print("Mean Cross-Validation, Kfold: {:.3f}".format(np.mean(cross_val_score(lasso, X_train, y_train))))

lasso_unscaled = np.mean(cross_val_score(lasso, X_train, y_train)) # Will use later to compare results

LASSO REGRESSION (UNSCALED DATA)
Training set score: 0.387
Test set score: -0.730
Mean Cross-Validation, Kfold: 0.053
